## 1. 다음영화 - 이미지 크롤링

- 2020년도 영화순위 이미지 가져오기
- res = requests.get(url)
- res.text : 문자열 형식으로 응답 content 추출
- res.content : 바이트열 형식을 응답 content 추출

In [15]:
import requests
from bs4 import BeautifulSoup


url = 'https://search.daum.net/search?w=tot&q=2020%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

imgs = soup.find_all('img', attrs={'class':"thumb_img"})

for idx, img in enumerate(imgs):
    img_url = img['src']
    #print(img_url)
    
    imgs_res = requests.get(img_url)
    imgs_res.raise_for_status()
    
    with open('data_imgs/movie1/movie{}.jpg'.format(idx+1), 'wb') as f:
        f.write(imgs_res.content)

- 2020년 상위 5개 이미지만 가져오기

In [20]:
for idx, img in enumerate(imgs):
    img_url = img['src']
    #print(img_url)
    
    imgs_res = requests.get(img_url)
    imgs_res.raise_for_status()
    
    with open('data_imgs/movie2/movie{}.jpg'.format(idx+1), 'wb') as f:
        f.write(imgs_res.content)
        
    if idx >= 4: break

- 2015~2020년까지 상위 5개 영화 가지고 오기

In [76]:
count = 0
for i in range(2015, 2021):
    url = f'https://search.daum.net/search?w=tot&q={i}%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')

    imgs = soup.find_all('img', attrs={'class':"thumb_img"})

    for idx,img in enumerate(imgs):
        img_url = img['src']
    
        imgs_res = requests.get(img_url)
        imgs_res.raise_for_status()
    
        with open('data_imgs/movie3/movie{}-{}.jpg'.format(i, idx+1), 'wb') as f:
            f.write(imgs_res.content)
            
        if idx >= 4:
            break
        count+=1

## 2. 코스피 시가총액 순위

In [104]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook


url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page='
wb = Workbook()
ws = wb.active
title = 'N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE'.split('\t')
ws.append(title)

for page in range(1, 5):
    res = requests.get(url+str(page))
    soup = BeautifulSoup(res.text, 'lxml')
    
    data_rows = soup.find('table', class_="type_2").find('tbody').find_all('tr')
    
    for row in data_rows:
        cols = row.find_all('td')
        if len(cols) == 1:
            continue
        data = [col.get_text().strip() for col in cols]
        data.pop(-1)
        ws.append(data)
        
wb.save('excel/시가총액.csv')

## 3. 과제
### 3-1) 네이버에서 오늘 광주(경기도)의 날씨정보를 가져오기
[오늘의 날씨]   
맑음, 어제보다 4˚ 높아요   
현재 온도 :27℃ (최저 21˚/최고 27˚)   
강수확률 : 오전강수확률 30% / 오후강수확률 20%   
미세먼지 : 29㎍/㎥좋음   
초미세먼지 : 20㎍/㎥보통   
오존지수 : 0.064ppm보통   


In [32]:
import requests
from bs4 import BeautifulSoup

url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EB%84%A4%EC%9D%B4%EB%B2%84+%EC%98%A4%EB%8A%98%EC%9D%98+%EB%82%A0%EC%94%A8&oquery=%EB%84%A4%EC%9D%B4%EB%B2%84+%EB%82%A0%EC%94%A8&tqi=hf3LXdprvxZssvLHjO0ssssst8K-303140'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

print('[오늘의 날씨]')
today_weather_talk = soup.find('p', attrs={'class':'cast_txt'})
print(today_weather_talk.get_text())

temperature = soup.find('p', attrs={'class':'info_temperature'}).get_text().replace('도씨', '')
min_temperature = soup.find('span', class_='min').get_text()
max_temperature = soup.find('span', class_='max').get_text()
print(f'현재온도:{temperature}(최저{min_temperature}/최고{max_temperature})')

morning_rain_rate = soup.find('span', class_='point_time morning').find('span', class_='rain_rate').get_text()
after_rain_rate = soup.find('span', class_='point_time afternoon').find('span', class_='rain_rate').get_text()
print(f'강수확률 : 오전{morning_rain_rate.strip()} / 오후{after_rain_rate.strip()}')
    
# 초미세먼지 결과가 측정기 없음좋음으로 뜰 경우
dust_and_ozone = soup.find_all('dl', attrs={'class':'indicator'})
for dao in dust_and_ozone:
    data = dao.get_text().split()
    if len(data) == 7:
        print(data[0],':',data[1])
        print(data[2],':',data[3], data[4])
        print(data[5],':',data[6])
    else:
        print(data[0],':',data[1])
        print(data[2],':',data[3])
        print(data[4],':',data[5])

[오늘의 날씨]
맑음, 어제보다 4˚ 높아요
현재온도:21℃ (최저19˚/최고24˚)
강수확률 : 오전강수확률 30% / 오후강수확률 30%
미세먼지 : 12㎍/㎥좋음
초미세먼지 : 1㎍/㎥좋음
오존지수 : 0.02ppm좋음


### 3-2) 네이버뉴스->it/과학->it일반 분야의 상위 3개 뉴스 크롤링 하기

[IT 뉴스]
1. 피 안 보고 혈당 측정
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=032&aid=0003095126 )
2. 갤폴드3 중고로 샀는데 “반납하세요” 문자…황당 사연, 무슨일?
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=016&aid=0001880910 )
3. 中, 미성년자 온라인게임 이용 주당 3시간으로 제한
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=092&aid=0002232128 )

In [40]:
import requests
from bs4 import BeautifulSoup


url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'lxml')

headlines = soup.find_all('dt')

head_list = []

for headline in headlines:
    headline = headline.get_text().strip()
    if headline != '':
        head_list.append(headline)

links = soup.find_all('dt', attrs={'class':'photo'})

for idx, link in enumerate(links):
    if idx >= 3:
        break
    print(idx+1, head_list[idx], '(링크 : {})'.format(link.a['href']))


1 "AI 적용할 1순위는 의료산업" (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=015&aid=0004598803)
2 KT 원팀, 초거대AI 내년 상반기 상용화 (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=015&aid=0004598797)
3 CJ올리브, 인공지능 산학협력 광주과기원에 제2연구소 설립 (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=421&aid=0005577403)


In [24]:
def create_soup(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup


In [33]:
def print_news(index: int, title, link):
    print('{} {}'.format(index+1, title))
    print('(링크 : {} )'.format(link))

In [35]:
def scrape_news():
    print('[IT 뉴스]')
    url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230'
    soup = create_soup(url)
    news_list = soup.find('ul', class_='type06_headline').find_all('li', limit=3)
    
    for idx, news in enumerate(news_list):
        a_idx = 0
        img = news.find('img')
        if img:
            a_idx = 1
        a_tag = news.find_all('a')[a_idx]
        title = a_tag.get_text().strip()
        link = a_tag['href']
        print_news(idx, title, link)
    
    
scrape_news()

[IT 뉴스]
1 국민지원금 사용처 '네이버·카카오 지도'에서 본다
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=008&aid=0004641159 )
2 "네이버·카카오맵서 코로나 국민지원금 사용처 확인하세요"(종합)
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=001&aid=0012644259 )
3 "추석 연휴에도 쏘카 타세요"···대여료 최대 78% 할인
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=011&aid=0003958938 )


### 3-3) 만개의 레시피 사이트에서 명예의 전당 상위 5페이지 레시피 정보(제목, 링크) 가지고 오기   
[명예의 전당]   
닭봉간장조림~ 십년째 만들어 먹는 양념 공유해요! https://www.10000recipe.com/recipe/6940325   
백파더 백종원 에그치즈토스트 ~ 간단한데 맛은 최고! https://www.10000recipe.com/recipe/6939543   
..생략....

In [21]:
import requests
from bs4 import BeautifulSoup


main_titles = soup.find_all('div', class_='tag_tit')

for main in main_titles:
    main = main.a.find_next_sibling('a').get_text().split()
    print(f'[{main[1]}]')
        
for i in range(1, 6):
    url = f'https://www.10000recipe.com/issue/view.html?cid=9999scrap&page={i}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    
    titles = soup.find_all('a', class_="thumbnail")
    
    for title in titles:
        print(title.get_text().strip(), 'https://www.10000recipe.com'+title['href'])

    

[명예의전당]
닭봉간장조림~ 십년째 만들어 먹는 양념 공유해요! https://www.10000recipe.com/recipe/6940325
백파더 백종원 에그치즈토스트 ~ 간단한데 맛은 최고! https://www.10000recipe.com/recipe/6939543
♥[신혼밥상] 5분 반찬 상추겉절이 https://www.10000recipe.com/recipe/6929388
집밥으로 술안주로~ 닭갈비 레시피 끝판왕!! 파는 닭갈비 비법을 담았슴돠ㅋ~^^ https://www.10000recipe.com/recipe/6928209
즉석 목살 양념구이~ 달큰한 양념소스가 정말 맛있습니다. https://www.10000recipe.com/recipe/6925795
[아이반찬] 꼬들꼬들 식감이 좋은 맵지않은 오이무침 https://www.10000recipe.com/recipe/6923484
차돌박이 된장찌개 레시피ㅣ고깃집에서 먹던 바로 그 맛!ㅣ https://www.10000recipe.com/recipe/6922749
새우 부추전ㅣ집들이용으로도 손색 없는! https://www.10000recipe.com/recipe/6922700
치킨 스테이크 만들기ㅣ굽네가 부럽지 않은ㅣ https://www.10000recipe.com/recipe/6922190
간단한 술안주로 최고인 버터갈릭새우 만들기 https://www.10000recipe.com/recipe/6918977
맛있는 밑반찬 백종원 가지볶음 https://www.10000recipe.com/recipe/6917883
♥[신혼밥상] 10분 반찬! 메추리알조림 https://www.10000recipe.com/recipe/6916408
밥도둑 반찬 고추장 달걀조림 만들기!TV 알토란 레시피 https://www.10000recipe.com/recipe/6915088
감자어묵국 보들보들하게 즐기는 간단국요리 https://www.10000recipe.com/recipe/6914749
버섯으로 관자 느낌